In [1]:
%pylab inline

import scipy.special as sp
from scipy.stats import nbinom
import scipy.stats

Populating the interactive namespace from numpy and matplotlib


# The Ship of Theseus: Manufacturing Errors

There are multiple ways to build and maintain large, complex systems in space. There's the traditional idea of "build it on the ground, launch it into orbit", which has been the established way of doing things since the 60s. Recently, technologies like 3d printing and robotic assembly have established a continuum of solutions, each with varying levels of component and contruction complexity.

Here's a picture a labmate, Benjamin Jenett, made that captures this continuum.

<img src="https://raw.githubusercontent.com/dcellucci/Research_Reports/master/Manufacturing_Errors/figures/pva.JPG" width=40%/>

As you can see, there's a wide range of solutions distributed along a Pareto front, which are differentiated by how complicated their parts are, and how complicated the machines that build them are. A deployable has the simplest assembler, since there isn't any assembly being done at all, but instead the part itself is composed of complex biphase mechanisms that allow it to enter a mission configuration once it reaches the target environment. These mechanisms make for an extremely complicated part, however. On the flip side, in-Situ fabrication is essentially 3d printing, and involves a simple part type (plastic or a polymer of some type) but a very complex assembler. 

(You can also think of this in the context of manufacturing spares for an ECLS system, where the part is the material which composes the spare, and the assembler is the machine which constructs the spare from the parts.)

The point is that there's a whole range of solutions that can exist between these two extremes, where the parts that are being built are maybe not 

The above figure is, however, a qualitative graph illustrating the idea. What doesn't exist right now is a method for quantitatively comparing these various solutions. I should make it clear at this point that don't think that it will ever be as simple as a single Pareto front- the relative advantages of each process become apparent in different contexts- but I do think that there is a simple way to characterize these advantages so that we can identify a

And just to be clear, here's the list of manufacturing processes I decided to look at:

  1. **In-Situ Fabrication:** 3D Printing
  2. **On-Orbit Assembly:** Work like welding robots, or SpiderFab. Anywhere where a truss structure is being built out of non-reversible connections
  3. **Digital Materials:** All parts are mechanically connected, and the energy required to remove a component is approximately equal to the energy required to place it. 
  4. **Deployables:** Most things in space right now. 

## Repairability

The differentiating characteristic of these processes that I decided to use was the repairability or fault-tolerance of the processes. In order to talk about repairability, I broke a manufacturing process into a sequence of discrete steps, each of which has a probability of failure $p$. What is a failure, exactly? Well...

   > A failure is defined as an event that decreases the functionality of the part so much that it needs to be addressed, either in the form of repair, or complete restarting of the process (and waste of the material used so far to make the part). 
    
The processes I described above fall into one of three categories, depending on how they respond to failure. We have:

  1. **Non-repairable:** where a single failure means that the process has to be restarted. 
  2. **Fault-Tolerant:** where the process can accomodate a specific number of failures before the process needs to be restarted. This can be accomplished through something like built-in redundancies or patching. 
  3. **Reversible:** The part is the Ship of Theseus- it can accomodate effectively infinite replacements with no significant degradation of the overall quality (other than the energy required to do so). Think Lego.
  
I'm going to show that, if you make a large enough object such that the number of manufacturing steps approaches the number of base-pairs in DNA or the number of transistors in a commercial CPU or a 1 km x 1 km x 10 m square plate made out of 1 m struts, you either need heirarchical fault-tolerance (a fault-tolerant structure made out of fault-tolerant parts) or you need reversibility in your manufacturing process. 

I'm also going to assume that in all of these cases, there is some form of per-step inspection. That is, the manufacturing process has some way of knowing whether or not a failure has occured immediately after it happens. This becomes important when you talk about adressing failures, since in many processes, the failure could be in a place that requires the part to be disassembled in order to reach that spot for repairs.

## Non-Repairable Processes

This is all deployables, 3d Printing. 

An example of a failure in a deployable is the P6 SAW redeployment on the ISS during STS-120, where a single micrometeoroid impact caused a tear in the structure which required manual repair by an astronaut. This sort of repair was unexpected and dangerous- the voltage of the array was enough to kill the astronaut if he'd made a ground path with the electrical system. As a result, this not a repairable process since a single failure requires putting an astronaut in a near-fatal position in order to perform a repair.

<img src="figures/tear.png" width=60%/>

### The Math

We can treat something like the redeployment of a solar array wing as a sequence of binary events, success or failure, and as a result the probability of a successful deployment $s$ (i.e. one with no failures) is the probability of every step occuring successfully. That's equivalent to, given a process with $n$ steps, each of which with failure probability $p$

$$
s = (1-p)^n
$$

The solar array blanket consists of 100 folds, each of which has to deploy without tearing in order to deploy the structure. If we want a desired success rate $s$, we can calculate the necessary per-fold failure probability. 

$$
 p = 1-s^{1/n}
$$

Where $n = 100$. Therefore, if we want a 99% success rate, the per-fold failure probability needs to be 

$$
 p = 1-0.99^(0.01) \approx 1\times10^{-4}
$$

Which is not unreasonable for a mechanical process, especially one that has to be performed only once. The difficulty comes when the array has been sitting open in LEO orbit for **seven years**, being cooked by radiation and bombarded with micrometeoroids. This was the case with the P6 truss- it was originally placed at the Z1 node in late 2000 during STS-97 and then was moved in 2007 during STS-120 to the end of the port ITS.

Let's be generous and say that those seven years increases the unfolding failure probability by an order of magnitude, so that $p=0.001$. The resulting success probability of the deployment is now

$$
 p = (1-0.001)^{100} \approx 0.905
$$

Which is still a good chance but maybe not good enough to trust an astronaut's life to it. If it was $p=0.01$, the probability of success falls to

$$
 p = (1-0.01)^{100} \approx 0.366
$$

Which is definitely unacceptable. 

## Fault-Tolerant Processes

We can also think of these as redundant processes, where individual failures are only incremental losses to system functionality, but beyond a certain point you either run out of redundancies and you have to do something drastic to repair the system.

I mentioned how one failure in the deployment mechanism required EVA repair. Compare this to the electrical system topology of the same solar arrays, which use bypass diodes to prevent power loss in the event of individual panel failure. Here, a micrometeroid has impacted a diode, and only the affected panel fails (due to overheating). This design is fault-tolerant, since, if enough diodes fail the system overall will fail, but multiple individual diodes can fail with only incremental loss of system functionality. 

<img src="figures/bypass.png" width=40%/> 

The actual fault-tolerance of the solar array wiring is hard to quantify since the threshold beyond which the array cannot supply enough power to be useful depends on a variety of factors. However, one important thing to note is that, unlike the mechanical aspects of the array, which consisted of 100 moving parts (more or less), the electrical system consists of 32,800 cells connected together. It simply isn't reasonable to expect a system of that size, where any single component can cause a total system failure, to function for an extended period of time. 

A related industry that operates a much larger scales is semiconductor fabrication, such as in the manufacture of L3 cache memory in commercial CPUs. L3 cache memory is essentially a 2D square grid of transistors, each of which is connected to its immediate neighbors. A failure of any one of these transistors means that the entire row or column is unusable. To combat this, usually these memory modules are manufactured with extra rows/columns such that the faulty rows/columns can be switched off and the desired number of bits will still be available. 

(I've yet to find a source that specifically lays out exactly how many extra rows/columes. This seems to be a low enough problem, and specific enough, that no one in this field has cared to talk about it an intentional way. The point is, we can answer that question now.)

Semiconductor production is dominated by yields, and the engineers who design the chips have adapted their architectures to accomodate error rates that would normally produce near-zero yield for the size chips they produce. Simply adding extra rows and columns stopped being a valid solution decades ago, and so these engineers accomplish reasonable yields by using a process I've termed *heirarchichal fault-tolerance*. 

### The Math

One cool thing we can do is estimate  the per-bit failure probability from an expected yield, as a result of the chosen fault-tolerant architecture. I'm pretty sure the engineers working right now at Intel don't know this information, but it's implicit in the choices they've made.

A fault-tolerant process is successful if a product has less than $m$ failures. That means those failures can be distributed among any of the $n$ steps, and the product is still considered successful. 

  > **Caveat:** A potential next step is to discuss error clusters, since groups of failures are going to be worse than ones that are spread out among the whole structure, or maybe failures cause more failures (such as a misalignment that causes subsequent misalignment), resulting in clusters. I haven't explored this.

Regardless, we can model the probability of a process creating a product with k errors using the Binomial Distribution:

$$
s_{ind}(k;n,p) = \begin{pmatrix} n \\ k \end{pmatrix} p^k (1-p)^{n-k}
$$

and since we allow any product with less than $m$ errors, we just sum all of the probabilities between 0 and $m$.

$$
s_{tot}(m;n,p) = \sum^{m+1}_{k=0}\begin{pmatrix} n \\ k \end{pmatrix} p^k (1-p)^{n-k}
$$

As you can see, the fault-tolerant process is the same as the non-repairable, if we assume m = 0. 

$$
s_{tot}(0;n,p) = \sum^1_{k=0}\begin{pmatrix} n \\ k \end{pmatrix} p^k (1-p)^{n-k} = (1-p)^n
$$

#### Example: The Itanium

For our specific example (the Itanium processor), the 24 MB L3 cache architecure is set up into twenty-four 1-MB ways, each of which contains 32 data subarrays with 1 redundancy. All 24 of these ways must function in order for the cache to work. In addition, a subarray is composed of approximately 114,285 bits, each of which must be functioning in order for the subarray to pass inspection.

If that previous paragraph confused you, trust me, the original white paper is worse. In our defined nomenclature, we can separate this into three different processes in order to produce a 24 MB cache, listed from highest level to lowest.
   
   1. A non-repairable process consisting of 24 steps, one for each way. 
   2. A fault-tolerant process consisting of 66 steps with a maximum of 2 faults, one for each half-subarray in a given way
   3. A non-repairable process consisting of 114,285 steps, one for each bit in a given half-subarray. 

If we assume that 24 MB subarrays have a 90% yield, which is an assumption we have to make because yields are a closely guarded secret in chip fab, then the code below generates the expected per-bit error rate for that yield.

In [186]:
#This code generates the values described above. 
#First, I played with values of p, the per-half-subarray failure rate, until the 
#yield (tot_prob) equaled ~90%. 
#I then took this value of p, and used it to calculate the per-bit-failure rate
#with fault-tolerance (pbf-wft) and without (pbf-nft).

p = 0.00494250967308 #half-subarray probability of failure
tot_prob = 0

for k in range(0,3):
    tot_prob = tot_prob + sp.binom(66,k)*p**k*(1-p)**(66-k)

tot_prob = tot_prob**24

pbf_wft = 1-(1-p)**(1.0/114285)
pbf_nft = 1-(0.9)**(1.0/(1.92*10**8))

print("               Yield: {0}\n".format(tot_prob))

print("      -Per-bit error rate-\nWith fault-tolerance: {0}".format(pbf_wft))

print("             Without: {0}\n".format(pbf_nft))

print("That is a {0}-fold increase in the acceptable error rate".format(int(pbf_wft/pbf_nft)))

               Yield: 0.9

      -Per-bit error rate-
With fault-tolerance: 4.33544573575e-08
             Without: 5.48752709939e-10

That is a 79-fold increase in the acceptable error rate


So that's an estimated per-bit error rate of $4.34\times10^{-8}$. Without fault-tolerance, the acceptable error rate is about 80 times lower, or $5.48\times10^{-10}$

There is an active question, at least to me, whether or not using this tool to design architectures would help chip manufacturers at all. Take, for instance, if we instead designed a shifting protocol such that we could replace any 24 errors with 24 redundancies. Ostensibly this would result in more overhead, but it would either

  1. approximately double the acceptable per-bit error rate and still produce 90% yield, allowing less-precise (and therefore cheaper) manufacturing processes. 
  2. Increase the existing yield from 90% to $(100-5.3\times10^{-15})\%$, which is a number hilariously close to 100%. 

In [185]:
#Here's where I back up the previous numbers

p = 0.011234 #per half-subarray error rate for fully-redundant architecture

tot_prob = 0
for k in range(0,25):
    tot_prob = tot_prob + sp.binom(1680,k)*p**k*(1-p)**(1680-k) 
    
pbf_wft = 1-(1-p)**(1.0/114285)
pbf_nft = 1-(0.9)**(1.0/(1.92*10**8))

print("               Yield: {0}\n".format(tot_prob))

print("      -Per-bit error rate-\nWith fault-tolerance: {0}\n".format(pbf_wft))

#Taking the previously calculated error rate (estimated from the described architecture)
p = 1-(1-0.00494250967308)**(1.0/114285)

tot_prob = 1
for k in range(0,25):
    tot_prob = tot_prob - sp.binom(1680,k)*pow(p,k)*(1.0-p)**(1680-k)
    
print("-Using previous per-bit error rate-\n           New Yield: 1.0-({0})".format(tot_prob))

               Yield: 0.900011364062

      -Per-bit error rate-
With fault-tolerance: 9.88544203162e-08

-Using previous per-bit error rate-
           New Yield: 1.0-(5.29570467166e-17)


As chip-manufacturers find it harder and harder to reach high yields at the desired length scales, they might want to turn to optimizing their fault-tolerant architectures to instead efficiently produce their chips.

## Reversible

Anyway, that extended aside was to illustrate that 1) fault-tolerance/redundancy is used extensively right now in some of the most demanding engineering environments and 2) we can still do better there, and perhaps we now have a way of assessing whether or not a given architecture is worth the overhead.

Speaking of which, the final type of manufacturing process is one that has historically been criticized for having way too much overhead. I'm going to attempt to show that the reversibility can be worth the extra effort once you find yourself building something sufficiently large.

To do this, I need to come up with some way to compare these processes that don't use yield. This is because there's really no such thing as yield with a reversible process, since it's possible to keep repeating failed steps until a perfect product is reached. 

But we'll still have to expend resources, material, energy, time, to produce these products. Therefore, I'll introduce a concept called the *Expected Resource Expenditure* or ERE. For a given manufacturing process requiring $n$ steps, the ERE is an estimate of how many steps will actually be performed in order to create that product.

With a reversible scheme, it we assume that each step has a failure probability $p$, we can expect that $np$ of these steps will fail and require replacement. If we then redo these failed steps, we will find that $(np)p$ of these steps have, in turn, failed, and so on, forming an infinite series

$$
n(p + p^2 + p^3 + \cdots) = n\frac{p}{1-p}
$$

This is the expected number of steps we would have to perform for a process with $n$ steps. We will define the ERE as the total cost to create the product, including the $n$ steps embodied in the product, normalized by the length of the manufacturing process. That is,

$$
R = \frac{1}{n}\left(n\frac{p}{1-p}+n\right) = \frac{1}{1-p}
$$

Which does not depend on the size of the process. What this means is that the ERE for a reversible process scales linearly with the number of steps. No matter how complex or large the project becomes, the overhead is going to be a fixed percentage of the size. 

### Comparing to Non-Repairable and Fault Tolerant Processes

How does this overhead compare to the previous two processes? Because we cannot treat each step individually as we did with the reversible scheme, we can instead calculate this as the number of times we have to perform a process before it produces a successful product. 

In this case, we are going to assume that we are doing this process enough times that the averages reasonably describe the actual behavior. That is, if you're going to build something that involves 3d printing 1000 parts, you can treat that statistically. If you're deploying 8 solar arrays, that's a bit harder to treat statistically. Since we want to build big things, or lots of things over a long period, I think the statistical treatment is appropriate.

The most naive solution is to simply take the yields we calculated above, assume that this represents an estimate of the number of times the process will have to be repeated before a functioning product is produced, and call that the overhead. This approach is what I would call the worst-case ERE, where the product can only be tested for functionality after it has been made. 

But what if we instead could prematurely end a process if it encounters a critical number of errors? Then the overall waste would be lower, since we wouldn't spend $n$ steps per failed part, but some other amount less than $n$. If we can check after every single step if it was successful, this would result in the best-case ERE, since no steps would be wasted on already failed parts. 

What we need to know in order to calculate this best-case ERE is what the average size of a failed part is. In order to find this average, we will use a negative binomial distribution.

#### Non-Repairable Processes
Remember that the previous probability of success $s$ we calculated beforehand was
$$
 s = (1-p)^n
$$

and therefore the worst case expected resource expenditure $R_{wc}$ is

$$
 R_{wc} = \frac{1}{n}\frac{n}{(1-p)^n}
$$

using a negative binomial distribution, we can find the distribution of discard sizes $k$ when the first ($r=1$ error occured

$$
d(k;r=1,p) = \begin{pmatrix} k \\ k \end{pmatrix}p^1(1-p)^k = p(1-p)^k
$$

however, this distribution also includes values of $k > n$, which doesn't make sense because we stop after $n$ steps. Therefore, the mean discard size $\bar{D}$ is the mean of the truncated distribution.

$$
\bar{D} = \sum^n_{k=0}(k+1)\ p\ (1-p)^k
$$

We can then multiply $R_{wc}$ by the factor $\bar{D}/n$ to get the best-case expected resource expenditure.

$$
 R_{bc} = \frac{\bar{D}}{n}\frac{1}{(1-p)^n}
$$

#### Fault-Tolerant Processes

Fault-tolerant processes follow a similar suit. Recall that our expected probability of success was the regularized incomplete beta function $I$

$$
s = I_{1-p}(n-m,1+m)
$$

and therefore the yield is

$$
R_{wc} = \frac{1}{I_{1-p}(n-m,1+m)}
$$

The average discard size here is the average size of a product, including previous errors, when the process reaches the $m^{\textrm{th}}$ error. This can be represented by the negative binomial distribution, where the average over the range $(m,n)$ gives the expected discard size:

$$
\bar{D} = \sum^n_{k=m}(k+1) \begin{pmatrix} k-1 \\ k-m \end{pmatrix} p^m (1-p)^{k-m}
$$

as a result, the best-case expected resource expenditure is calculated the same way as the non-repairable process

$$
R_{bc} = \frac{\bar{D}}{n}\frac{1}{I_{1-p}(n-m,1+m)}
$$

## Putting it All Together

I've made a chart that allows us to look at all three processes 

<img src="figures/big_chart.png"/>

Here, we have the $R_{bc}$ values for three different processes being shown in a variety of different conditions, as a function of manufacturing process length $N$. We have a non-repairable process for three different probabilities (solid lines), a fault-tolerant process with $p = 0.1$ and three different values for $m$ (dashed lines), and finally three different failure probabilities for a reversible process (dotted lines). 

What's important about this chart is that, for very large system sizes, the expected resource expenditure of the non-repairable and fault-tolerant schemes will experience exponential growth in the expected resources required to construct an object. 

I think this chart could be cleaned up. Namely, there is a scale-invariant factor that combines $N$, $m$, and $p$ in such a way that the six graphs composing the non-repairable and fault-tolerant processes become one. 

What this illustrates is that all $p$ does is affect the scale of the ERE for a given process, and therefore performance gains in $p$ are linear- a process that is 10 times better (less likely to fail) enables 10 times larger systems. 

Conversely, a fault-tolerant process shifts the ERE growth curve on the $N$-axis, but doesn't change the scaling. That's why the above fault-tolerant plots seem to get steeper and steeper as $m$ increases; the scaling is nearly the same as when $m=0$, but the regime where the scaling occurs is shifted over. 

What this means is that, for larger processes, it can be more effective to focus on designing a fault-tolerant architecture than to improve the fidelity of the underlying process. You can see this where the m=100 and the p=0.001 curves intersect in the $(10^2,10^3)$ range, where the fault-tolerant process has nearly perfect yield before hitting its asymptote. 

But regardless, the clear winner in this work is reversibility (surprise surprise). Unlike the other two manufacturing processes, the waste factor scales only linearly with system size, effectively sidestepping the combinatorial problems that the other two run into.  
